In [0]:
import os
os.getcwd()
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.stats import loguniform


In [0]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1-uuRcH-a-thRr6YQELzthhVo6FuKKVGI'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('stokes_triangle_1000.npz') 


file_id = '1y89bVug8-LRsl7hog8sJseyqq5HNn7a6'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('ResNet_18.py')

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'CLASSES/SP20/ece228/Project/Code/Networks/ResNet/doe_1_5_28/'
os.mkdir(root_dir + 'CLASSES/SP20/ece228/Project/Code/Networks/ResNet/doe_3_5_31/')
base_dir = root_dir + 'CLASSES/SP20/ece228/Project/Code/Networks/ResNet/doe_2_5_29/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data = np.load('stokes_triangle_1000.npz')


In [0]:
from ResNet_18 import ResNet,BasicBlock


In [0]:
for k in data.iterkeys():
    print(k)

sdf
vx
vy
rho


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: NpzFile.iterkeys is deprecated in python 3, to match the removal of dict.iterkeys. Use .keys() instead.
  """Entry point for launching an IPython kernel.


In [0]:
# swap the axes
sdfs = data['sdf']
sdfs = np.swapaxes(sdfs,0,2)
vx   = data['vx']
vx = np.swapaxes(vx,0,2)
vy   = data['vy']
vy = np.swapaxes(vy,0,2)
rho  = data['rho']
rho = np.swapaxes(rho,0,2)

# scale the data
sdf_scaler = StandardScaler()
vx_scaler  = StandardScaler()
vy_scaler  = StandardScaler()
rho_scaler = StandardScaler()

sdf_scaler.fit(sdfs.reshape((-1,1)))
vx_scaler.fit(vx.reshape((-1,1)))
vy_scaler.fit(vy.reshape((-1,1)))
rho_scaler.fit(rho.reshape((-1,1)))

sdfs_scaled = np.reshape((sdf_scaler.transform(sdfs.reshape((-1,1)))),sdfs.shape)
vx_scaled = np.reshape((vx_scaler.transform(vx.reshape((-1,1)))),vx.shape)
vy_scaled = np.reshape((vy_scaler.transform(vy.reshape((-1,1)))),vy.shape)
rho_scaled = np.reshape((rho_scaler.transform(rho.reshape((-1,1)))),rho.shape)


In [0]:
torch_sdf = torch.from_numpy(sdfs_scaled).float()
torch_vx  = torch.from_numpy(vx_scaled).float()
torch_vy  = torch.from_numpy(vy_scaled).float()
torch_rho = torch.from_numpy(rho_scaled).float()


In [0]:
tr_input = torch.unsqueeze(torch_sdf,dim=1)
print(tr_input.shape)
output = torch.stack((torch_vx,torch_vy,torch_rho),dim=1)
output.shape

torch.Size([1000, 1, 64, 128])


torch.Size([1000, 3, 64, 128])

In [0]:
data_set = torch.utils.data.TensorDataset(tr_input,output)
# split data set
train_set,val_set = torch.utils.data.dataset.random_split(data_set,(800,200))


In [0]:
# tr_loader = torch.utils.data.DataLoader(train_set)
# val_loader = torch.utils.data.DataLoader(val_set)

In [0]:
def hyp_tune_net(tr_set,val_set,lr,val_split=0,epochs=1,num_channels = 32,hyp_tune=True,
                 steps=1000,batch_size=20,run_num=0):
  tr_loss_vec = []
  val_loss_vec = []

  loss  = nn.MSELoss()
  block = BasicBlock
  model = ResNet(block = block,layers = [2,2,2,2],num_channels=num_channels)
  model.cuda()
  model.float()
  optim = torch.optim.Adam(model.parameters(),lr=lr)

  tr_loader = torch.utils.data.DataLoader(tr_set,batch_size = batch_size,
                                          shuffle=True,pin_memory = True)
  val_loader = torch.utils.data.DataLoader(val_set,batch_size = batch_size,
                                          shuffle=True,pin_memory = True)


  j = 1
  if hyp_tune==True:
    while j < steps:
      torch.set_grad_enabled(True)
      model.train()

      for idx,(data) in enumerate(tr_loader):
        x = data[0].cuda()
        # print(type(x))
        y = data[1].cuda()
        out = model(x)
        # print(y.shape,out.shape)
        sample_loss = loss(out,y)
        tr_loss_vec.extend([sample_loss.cpu().detach().numpy()])
        sample_loss.backward()
        optim.step()
        optim.zero_grad()
        j+=1
              
        if idx%10 == 0:
          print(idx)

  else:
    for i in range(epochs):


      for idx,(data) in enumerate(tr_loader):
        x = data[0].cuda()
        # print(type(x))
        y = data[1].cuda()
        out = model(x)
        # print(y.shape,out.shape)
        sample_loss = loss(out,y)
        tr_loss_vec.extend([sample_loss.cpu().detach().numpy()])
        sample_loss.backward()
        optim.step()
        optim.zero_grad()

        
        if idx%10 == 0:
          print(idx)

      if val_split is not 0:
        torch.set_grad_enabled(False)
        model.eval()

        for v_idx,(v_data) in enumerate(val_loader):
          x = v_data[0].cuda()
          y = v_data[1].cuda()
          out = model(x)
          sample_loss = loss(out,y)
          val_loss_vec.extend([sample_loss.cpu().detach().numpy()])

  hyp_dict = {}
  hyp_dict['tr_loss']  = tr_loss_vec
  hyp_dict['n_epochs'] = epochs
  hyp_dict['lr']       = lr
  hyp_dict['batch_size'] = batch_size
  hyp_dict['num_channels'] = num_channels
  torch.save(hyp_dict,base_dir+'run'+str(run_num)+'.pt')

      




In [0]:
for i in range(100):
  try:
    print('iterration',i)
    batch_size = np.random.randint(low=1,high=9)*8 
    lr = float(loguniform.rvs(10**-4, 10**-1, size=1)[0])
    num_channels = np.random.randint(2,17)*16

    hyp_tune_net(tr_set = train_set,val_set = val_set,lr=lr,val_split=0,num_channels=num_channels,
                epochs=1,batch_size=batch_size,run_num=i)
  except:
    pass



Streaming output truncated to the last 5000 lines.
20
30
40
0
10
20
30
40
0
10
20
30
40
0
10
20
30
40
0
10
20
30
40
0
10
20
30
40
0
10
20
30
40
0
10
20
30
40
0
10
20
30
40
0
10
20
30
40
0
10
20
30
40
0
10
20
30
40
0
10
20
30
40
0
10
20
30
40
0
10
20
30
40
0
10
20
30
40
iterration 15
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
iterration 16
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
0
10
iterration 17
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
1

In [0]:
val_set